In [14]:
import pandas as pd 
from selenium import webdriver
import time
import matplotlib.pyplot as plt

In [17]:
# Function to return game logs from season and amount of pages wanted (2019-20 and 50 pages, for example)

def nba_game_logs1(num_clicks):
    
    # Path to Driver 
    path = '/Users/willhanley/Desktop/chromedriver 3'
    driver = webdriver.Chrome(executable_path=path)
    
    # URL
    url = 'https://www.nba.com/stats/players/boxscores/?Season=2020-21&SeasonType=Regular%20Season'
    driver.get(url)
    
    # location of table, button to click to next page 
    table = driver.find_element_by_class_name('nba-stat-table__overflow').text.split('\n')
    next_page = driver.find_elements_by_xpath('/html/body/main/div/div/div[2]/div/div/nba-stat-table/div[3]/div/div/a[2]')

    # empty lists to be appended to 
    player_names = []
    player_stats = []
    
    # Loop through num_clicks times
    for _ in range(num_clicks):
        
        # Define the table we are scraping from 
        table = driver.find_element_by_class_name('nba-stat-table__overflow').text.split('\n')
        
        # append to player_names and player_stats lists
        for num, info in enumerate(table):
            if num == 0:
                continue
            else:
                if num % 2 == 1:
                    player_names.append(info)
                if num % 2 == 0:
                    player_stats.append([i for i in info.split(' ')])
                
        # Click to next page            
        driver.execute_script("arguments[0].click();", next_page[0])
        
        # Pause 5 seconds 
        time.sleep(4)
 
    # Return dataframe made from player_names and player_stats
    return pd.DataFrame({'Player': player_names,
                  'Team': [i[0] for i in player_stats],
                  'Match-up': [i[2]+i[3] for i in player_stats],
                  'Date': [i[4] for i in player_stats],
                  'W/L': [i[5] for i in player_stats],
                  'Min': [i[6] for i in player_stats],
                  'Pts': [i[7] for i in player_stats],
                  'FGM': [i[8] for i in player_stats],
                  'FGA': [i[9] for i in player_stats],
                  'FG%': [i[10] for i in player_stats],
                  '3PM': [i[11] for i in player_stats],
                  '3PA': [i[12] for i in player_stats],
                  '3P%': [i[13] for i in player_stats],
                  'FTM': [i[14] for i in player_stats],
                  'FTA': [i[15] for i in player_stats],
                  'FT%': [i[16] for i in player_stats],
                  'OREB': [i[17] for i in player_stats],
                  'DREB': [i[18] for i in player_stats],
                  'REB': [i[19] for i in player_stats],
                  'AST': [i[20] for i in player_stats],
                  'STL': [i[21] for i in player_stats],
                  'BLK': [i[22] for i in player_stats],
                  'TOV': [i[23] for i in player_stats],
                  'PF': [i[24] for i in player_stats],
                  '+/-': [i[25] for i in player_stats]})

In [18]:
# Function if player has a double double 

def double_double(row):
    val = 0
    if (row['Pts'] >= 10) & (row['AST'] >= 10) | \
    (row['Pts'] >=10) & (row['REB'] >= 10) | (row['AST'] >=10) & (row['REB'] >= 10):
        val += 1
    return val

# Function if player has a triple double 

def triple_double(row):
    val = 0
    if (row['Pts'] >= 10) & (row['AST'] >= 10) & (row['REB'] >=10) | \
    (row['Pts'] >=10) & (row['AST'] >= 10) & (row['STL'] >= 10) | \
    (row['Pts'] >=10) & (row['REB'] >= 10) & (row['BLK'] >= 10) | \
    (row['Pts'] >=10) & (row['AST'] >= 10) & (row['BLK'] >= 10):
        val += 1
    return val

In [19]:
twenty = nba_game_logs1(64)
twenty.head()

,Player,Team,Match-up,Date,W/L,Min,Pts,FGM,FGA,FG%,...,FT%,OREB,DREB,REB,AST,STL,BLK,TOV,PF,+/-
0,Stephen Curry,GSW,vs.TOR,01/10/2021,W,38,11,2,16,12.5,...,100,3,6,9,6,0,0,5,3,7
1,Draymond Green,GSW,vs.TOR,01/10/2021,W,29,10,3,8,37.5,...,66.7,1,8,9,10,0,1,3,1,-3
2,Kelly Oubre Jr.,GSW,vs.TOR,01/10/2021,W,29,12,5,14,35.7,...,100,3,3,6,3,0,1,1,1,-4
3,Kevon Looney,GSW,vs.TOR,01/10/2021,W,14,2,1,1,100,...,-,2,1,3,0,0,1,0,0,5
4,Mychal Mulder,GSW,vs.TOR,01/10/2021,W,11,0,0,1,0.0,...,-,0,3,3,0,0,0,1,1,5


In [20]:
twenty.to_csv('data/20-21_game_logs.csv',index=False)

In [8]:
# Read in game logs from 2016-2020

df1 = pd.read_csv('data/2016=2017stats.csv')
df2 = pd.read_csv('data/2017-2018stats.csv')
df3 = pd.read_csv('data/2018-2019stats.csv')
df4 = pd.read_csv('data/game_logs2019-2020.csv')

# Date column to datetime

df1['Date'] = pd.to_datetime(df1['Date'])
df2['Date'] = pd.to_datetime(df2['Date'])
df3['Date'] = pd.to_datetime(df3['Date'])
df4['Date'] = pd.to_datetime(df4['Date'])

# Create Season column

df1['Season'] = '2016-17'
df2['Season'] = '2017-18'
df3['Season'] = '2018-19'
df4['Season'] = '2019-20'

# Save index as date

df1.set_index('Date', inplace=True)
df2.set_index('Date', inplace=True)
df3.set_index('Date', inplace=True)
df4.set_index('Date', inplace=True)

# Concat 1 dataframe
frames = [df1,df2,df3,df4]
game_logs_total = pd.concat(frames)
game_logs_total.sort_index(inplace=True)

# Drop columns with percentages

game_logs_total.drop(columns=['FT%', '3P%', 'FG%'], inplace=True)

# Save numerical columns as ints

game_logs_total[['Min', 'Pts', 'FGM','FGA','3PM', '3PA', 'FTM', 'FTA', 'OREB',
       'DREB','REB','AST','STL','BLK','TOV','PF', '+/-']] = game_logs_total[['Min', 'Pts', 'FGM','FGA','3PM', '3PA', 'FTM', 'FTA', 'OREB',
       'DREB','REB','AST','STL','BLK','TOV','PF', '+/-']].astype(str).astype(int)

# Create Double double column
game_logs_total['dubdub'] = game_logs_total.apply(double_double, axis=1)

# Create Triple double column
game_logs_total['tripdub'] = game_logs_total.apply(triple_double, axis=1)

# Create Draft kings score column (based on draftkings formula)
game_logs_total['DraftKings'] = game_logs_total['Pts'] + .5*game_logs_total['3PM'] + 1.25*game_logs_total['REB'] + 1.5*game_logs_total['AST'] \
+ 2*game_logs_total['STL'] + 2*game_logs_total['BLK'] -.5*game_logs_total['TOV'] + 1.5*game_logs_total['dubdub'] + 3*game_logs_total['tripdub']

# Save as csv
game_logs_total.to_csv('data/game_logs_total.csv')